In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import umap
import datetime
import optuna

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, label_binarize, PolynomialFeatures, RobustScaler
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.base import clone
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score, accuracy_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer

import statsmodels.api as sm

from scipy import stats

import xgboost

import gc

gc.collect()


3000

In [2]:
train_df = pd.read_csv("data/train.csv", index_col=0)
test_df = pd.read_csv("data/test.csv", index_col=0)
original_df = pd.read_csv("data/original.csv",sep=";")
train_features = test_df.columns

cat_features = ['Marital status', 'Application mode', 'Course',
                'Previous qualification', 'Nacionality', "Mother's qualification", 
                "Father's qualification", "Mother's occupation",
                "Father's occupation"]
cont_features = list(set(train_features).difference(cat_features))

In [3]:
for feat in cat_features:
    dtype = pd.CategoricalDtype(categories=list(set(train_df[feat]) | set(test_df[feat]) | set(original_df[feat])), ordered=False)
    for df in [train_df, test_df, original_df]:
        df[feat] = df[feat].astype(dtype)

In [4]:
for df in [train_df, test_df, original_df]:
    df["Misery Index"] = df["Unemployment rate"] + df["Inflation rate"]
    # df["Economic Discomfort"] = df["Misery Index"] - df["GDP"]

cont_features.append("Misery Index")

In [27]:
feature_importances_order = pd.DataFrame.from_dict(
    {
        "Curricular units 2nd sem (approved)": 0.5262597,
        "Tuition fees up to date": 0.12040115,
        "Scholarship holder": 0.04374109,
        "Curricular units 1st sem (approved)": 0.043124773,
        "Curricular units 2nd sem (enrolled)": 0.034864154,
        "Curricular units 2nd sem (evaluations)": 0.033007413,
        "Curricular units 1st sem (evaluations)": 0.026952056,
        "Debtor": 0.02125777,
        "Curricular units 2nd sem (grade)": 0.017488716,
        "Gender": 0.015914941,
        "Age at enrollment": 0.009499201,
        "Daytime/evening attendance": 0.009322386,
        "Curricular units 1st sem (enrolled)": 0.008687382,
        "Curricular units 2nd sem (credited)": 0.008607062,
        "Course": 0.008552427,
        "Mother's occupation": 0.0074276286,
        "Application mode": 0.007248743,
        "GDP": 0.0058370973,
        "Curricular units 1st sem (grade)": 0.00579759,
        "Unemployment rate": 0.0038729862,
        "Mother's qualification": 0.0035413294,
        "Displaced": 0.0034240438,
        "Previous qualification": 0.0033880775,
        "Father's occupation": 0.0033660706,
        "Curricular units 1st sem (credited)": 0.0032424054,
        "Admission grade": 0.0031858524,
        "Curricular units 1st sem (without evaluations)": 0.0031376902,
        "Marital status": 0.0027686018,
        "Father's qualification": 0.002645827,
        "Nacionality": 0.0026041167,
        "Application order": 0.0025012011,
        "Previous qualification (grade)": 0.0024294835,
        "Inflation rate": 0.0023359724,
        "International": 0.0020073373,
        "Curricular units 2nd sem (without evaluations)": 0.0015576994,
    },
    orient="index",
    columns=["val"],
)

low_freq_cols = [
    "Nacionality",
    "Educational special needs",
    "International",
    "Curricular units 1st sem (credited)",
    "Curricular units 1st sem (without evaluations)",
    "Curricular units 2nd sem (credited)",
    "Curricular units 2nd sem (without evaluations)",
]


# class PolynomialFeatures_DF(object):
#     def __init__(
#         self, degree=2, *, interaction_only=False, include_bias=True, order="C"
#     ):
#         self.poly_feats = PolynomialFeatures(
#             degree,
#             interaction_only=interaction_only,
#             include_bias=include_bias,
#             order=order,
#         )

#     def fit(self, X, y=None):
#         return self.poly_feats.fit(X)

#     def transform(self, X):
#         X_poly = self.poly_feats.transform(X)
#         col_names = self.poly_feats.get_feature_names_out()

#         return pd.DataFrame(X_poly, columns=col_names)

#     def get_params(self, deep=True):
#         return self.poly_feats.get_params(deep=deep)

class ObjectiveCV(object):
    def __init__(self, train_set, orig_set, kfold, n_jobs, random_state=0):
        self.train = train_set
        self.orig = orig_set
        self.kfold = kfold
        self.n_jobs = n_jobs

        self.random_state = random_state
        self.label_encoder = LabelEncoder()

    def __include_orig(self, inp_df):
        return pd.concat([inp_df, self.orig], axis=0)

    def __prune_low_freq(self, inp_df):
        return inp_df.drop(columns=low_freq_cols, errors="ignore")

    def __prune_low_importance(self, trial, inp_df):
        prune_thresh = trial.suggest_float("prune_thresh", 0, 0.5, step=0.001)
        pruned_cols = list(
            feature_importances_order.loc[
                feature_importances_order["val"] < prune_thresh
            ].index
        )
        return inp_df.drop(columns=pruned_cols, errors="ignore")

    def __model_logreg(self, trial):
        penalty = trial.suggest_categorical("penalty", [None, "l1", "l2", "elasticnet"])
        logreg_c = trial.suggest_int("logreg_c", 1, 100)

        if penalty == "elasticnet":
            l1_ratio = trial.suggest_float("l1_ratio", 0, 1)
            clf = LogisticRegression(
                random_state=self.random_state,
                n_jobs=self.n_jobs,
                penalty=penalty,
                C=logreg_c,
                l1_ratio=l1_ratio,
                solver="saga",
            )

        else:
            clf = LogisticRegression(
                random_state=self.random_state,
                n_jobs=self.n_jobs,
                penalty=penalty,
                C=logreg_c,
                solver="saga",
            )

        return clf

    def __model_rf(self, trial):
        # bootstrap = trial.suggest_categorical("bootstrap", [True, False])
        # max_sample = None
        # if bootstrap:
        #     max_sample = trial.suggest_float("max_samples", 0, 1)

        return RandomForestClassifier(
            random_state=self.random_state,
            n_jobs=self.n_jobs,

            n_estimators=trial.suggest_int("n_estimators", 100, 4e3),
            max_depth=trial.suggest_int("max_depth", 3, 10),
            min_samples_split=trial.suggest_float("min_samples_split", 0, 0.5),
            min_samples_leaf=trial.suggest_float("min_samples_leaf", 0, 0.3),
            min_weight_fraction_leaf=trial.suggest_float("min_weight_fraction_leaf", 0, 0.3),
            max_features=trial.suggest_float("max_features", 0, 1),
            # max_leaf_nodes=trial.suggest_int("mlf", 0, 5e3),
            # min_impurity_decrease=trial.suggest_float("min_impurity_decrease", 0, 5e3),
            # bootstrap=bootstrap,
            # warm_start=trial.suggest_categorical("warm_start", [True, False]),
            # max_samples=max_sample,
            # ccp_alpha=trial.suggest_float("ccp_alpha", 0, 5e3),
        )

    def __model_xgb(self, trial):
        return xgboost.XGBClassifier(
            enable_categorical=True,
            random_state=self.random_state,
            n_jobs=self.n_jobs,

            n_estimators=trial.suggest_int("n_estimators", 1, 1e4),
            eta=trial.suggest_float("eta", 0.01, 1),
            gamma=trial.suggest_float("gamma", 0, 20),
            max_depth=trial.suggest_int("max_depth", 3, 10, step=1),
            max_leaves=trial.suggest_int("max_leaves", 0, 1e4),
            colsample_bytree=trial.suggest_float("colsample_bytree", 0.05, 1),
            colsample_bylevel=trial.suggest_float("colsample_bylevel", 0.05, 1),
            colsample_bynode=trial.suggest_float("colsample_bynode", 0.05, 1),
            reg_lambda=trial.suggest_int("reg_lambda", 0, 20),
            reg_alpha=trial.suggest_int("reg_alpha", 0, 20),
            grow_policy=trial.suggest_categorical(
                "grow_policy", ["depthwise", "lossguide"]
            ),
            min_child_weight=trial.suggest_float("min_child_weight", 0, 1e3),
            max_delta_step=trial.suggest_float("max_delta_step", 0, 1e2)
        )

    def __aug_rfe(self, trial, steps, clf, total_feats, use_rfe):
        if use_rfe:
            n_features_to_select = trial.suggest_float("rfe_n_features_to_select", 0, 1)
            if int(n_features_to_select * total_feats) <= 0:
                n_features_to_select = 1

            steps.append(
                (
                    "model",
                    RFE(
                        clf,
                        n_features_to_select=n_features_to_select,
                        # step=trial.suggest_float("rfe_step", 0, 1),
                    ),
                )
            )
        else:
            steps.append(("model", clf))

    def __preproc_polyfeats(self, trial, low_degree=1, high_degree=2):
        return PolynomialFeatures(
            trial.suggest_int("poly_feats_degrees", low_degree, high_degree)
        )

    def __preproc_standardscaler(self):
        return StandardScaler()

    def __preproc_robustscaler(self, trial):
        return RobustScaler(
            with_centering=trial.suggest_categorical("with_centering", [True, False]),
            with_scaling=trial.suggest_categorical("with_scaling", [True, False]),
        )

    def preprocess(self, trial, inp_df):
        if trial.suggest_categorical("include_orig", [True, False]):
            inp_df = self.__include_orig(inp_df)
        if trial.suggest_categorical("prune_low_freq", [True, False]):
            inp_df = self.__prune_low_freq(inp_df)
        if trial.suggest_categorical("prune_low_importance", [True, False]):
            inp_df = self.__prune_low_importance(trial, inp_df)

        x = inp_df.drop(columns=["Target"])
        y = self.label_encoder.fit_transform(inp_df["Target"])

        remaining_feats = set(x.columns)
        cont_cols = [
            x.columns.get_loc(c) for c in remaining_feats.difference(cat_features)
        ]
        cat_cols = [
            x.columns.get_loc(c) for c in remaining_feats.difference(cont_features)
        ]

        numerical_pipeline = Pipeline([])

        if trial.suggest_categorical("poly_feats", [True, False]):
            poly_feats = self.__preproc_polyfeats(trial)
            numerical_pipeline.steps.append(("poly_feats", poly_feats))

        if trial.suggest_categorical("use_standardscaler", [True, False]):
            standard_scaler = self.__preproc_standardscaler()
            numerical_pipeline.steps.append(("standard_scaler", standard_scaler))

        if trial.suggest_categorical("use_robustscaler", [True, False]):
            robust_scaler = self.__preproc_robustscaler(trial)
            numerical_pipeline.steps.append(("robust_scaler", robust_scaler))

        col_transformer = None
        if len(numerical_pipeline.steps) > 0:
            col_transformer = ColumnTransformer(
                [("numerical_pipeline", numerical_pipeline, cont_cols)],
                remainder="passthrough",
            )

        if col_transformer:
            x = col_transformer.fit_transform(x)

        return x, y

    def init_model(self, trial):
        clf_selector = {
            "logreg": self.__model_logreg,
            "random_forest": self.__model_rf,
            "xgboost": self.__model_xgb,
        }

        return clf_selector[
            trial.suggest_categorical(
                "classifier", ["logreg"]
            )
        ](trial)

    def __call__(self, trial):
        train_df = self.train.copy()

        x, y = self.preprocess(trial, train_df)
        model = self.init_model(trial)

        scores = cross_val_score(
            model,
            x,
            y,
            scoring=make_scorer(accuracy_score),
            cv=self.kfold,
            n_jobs=self.n_jobs,
            error_score="raise",
        )

        return np.mean(scores)

In [28]:
crossval_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# optuna.delete_study(study_name="aug_rf_v1", storage="sqlite:///optuna.sqlite3")

# study = optuna.create_study(
#     direction="maximize",
#     storage="sqlite:///optuna.sqlite3",
#     study_name="classifiers_simple",
#     load_if_exists=True
# )

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///optuna.sqlite3",
    study_name="aug_logreg_v2",
    load_if_exists=True
)
objective = ObjectiveCV(train_df, original_df, crossval_kf, n_jobs=8)

study.optimize(objective, n_trials=50)
gc.collect()

[I 2024-06-17 17:43:13,144] Using an existing study with name 'aug_logreg_v2' instead of creating a new one.
[I 2024-06-17 17:44:28,699] Trial 54 finished with value: 0.8158342097745426 and parameters: {'include_orig': False, 'prune_low_freq': False, 'prune_low_importance': False, 'poly_feats': True, 'poly_feats_degrees': 2, 'use_standardscaler': False, 'use_robustscaler': False, 'classifier': 'logreg', 'penalty': 'l1', 'logreg_c': 28}. Best is trial 40 with value: 0.8158342097745426.
[I 2024-06-17 17:45:45,128] Trial 55 finished with value: 0.8158342097745426 and parameters: {'include_orig': False, 'prune_low_freq': False, 'prune_low_importance': False, 'poly_feats': True, 'poly_feats_degrees': 2, 'use_standardscaler': False, 'use_robustscaler': False, 'classifier': 'logreg', 'penalty': 'l1', 'logreg_c': 33}. Best is trial 40 with value: 0.8158342097745426.
[I 2024-06-17 17:47:00,381] Trial 56 finished with value: 0.8158342097745426 and parameters: {'include_orig': False, 'prune_low_f

KeyboardInterrupt: 

In [ ]:
study.best_params

{'include_orig': False,
 'prune_low_freq': False,
 'prune_low_importance': True,
 'prune_thresh': 0.011,
 'classifier': 'logreg',
 'penalty': 'l2',
 'logreg_c': 7}

In [ ]:
# best_model = make_pipeline(
#                 StandardScaler(),
#                 xgboost.XGBClassifier(enable_categorical=True, random_state=0, n_jobs=-1,
#                     eta=study.best_params["eta"],
#                     gamma=study.best_params["gamma"],
#                     max_depth=study.best_params["max_depth"],
#                     col_sample_bytree=study.best_params["col_sample_bytree"],
#                     col_sample_bylevel=study.best_params["col_sample_bylevel"],
#                     col_sample_bynode=study.best_params["col_sample_bynode"],
#                     reg_lambda=study.best_params["reg_lambda"],
#                     reg_alpha=study.best_params["reg_alpha"],
#                     grow_policy=study.best_params["grow_policy"],
#                     device="cuda"
#                 )
#             )

# label_encoder = LabelEncoder()
# targets = label_encoder.fit_transform(train_df["Target"])
# orig_targets = label_encoder.transform(original_df["Target"])

# combined_features = pd.concat([train_df[train_features], original_df[train_features]], axis=0)
# combined_targets = np.hstack([targets, orig_targets]) 

# best_model.fit(combined_features, combined_targets)
# test_preds = best_model.predict(test_df[train_features])

# out_pd = test_df.copy(deep=True)
# out_pd["Target"] = label_encoder.inverse_transform(test_preds)

# out_pd.to_csv("optuna_model.csv", columns=["Target"], index=True)

In [ ]:
# importances = best_model[1].feature_importances_
# rf_importances = pd.Series(importances, index=train_features)
# rf_importances = rf_importances.sort_values(ascending=False)
# rf_importances = rf_importances[rf_importances > 0]

# print(rf_importances.to_string())

# sns.barplot(x=rf_importances.index, y=rf_importances.values, hue=rf_importances.index, legend=False)
# plt.title('XGB Feature importances')
# plt.ylabel('Mean decrease in impurity')
# plt.xticks(rotation=45, ha="right")
# plt.show()

In [ ]:
# lenc = LabelEncoder()
# x = train_df.drop(columns=["Target"])
# y = lenc.fit_transform(train_df["Target"])

# test_trans = ColumnTransformer(
#     [("standardscaler", StandardScaler(), train_features)], remainder="passthrough"
# )
# pipeline = Pipeline(
#     [
#         (
#             "polyfeats_transformer",
#             ColumnTransformer(
#                 [
#                     (
#                         "polyfeats",
#                         PolynomialFeatures(2),
#                         x.columns,
#                     )
#                 ],
#                 remainder="passthrough",
#             ),
#         ),
#         # ("trans", test_trans),
#         # ("model", xgboost.XGBClassifier()),
#     ]
# )

# print(pipeline)


# # cross_val_score(pipeline, x, y, scoring=make_scorer(accuracy_score), n_jobs=-1)
# pipeline.fit(x)
# pipeline[0].transformers[0][1].get_feature_names_out(x.columns)

In [ ]:
# x_train, x_val, y_train, y_val = train_test_split(train_df.drop(columns=["Target"]),  train_df["Target"])

# test_model = RandomForestClassifier(n_jobs=-1)
# test_model.fit(x_train, y_train)
# preds = test_model.predict(x_val)
# sum(np.equal(preds, y_val)) / len(preds)